In [ ]:
import numpy as np 
import itertools as it
# from scipy.stats import multivariate_normal
from tqdm import tqdm
from scipy.stats import multivariate_normal
from scipy import sparse
from scipy.sparse import csr_matrix


t = np.arange(0,51,0.5)
e_x = np.linspace(-3,3,20)
e_y = e_x
th = np.linspace(-np.pi, np.pi,40)
X = list(it.product(t,e_x,e_y, th))
n_states = len(X)
state_table = {}
for i in range(n_states):
    state_table.update({X[i]:i})
v = np.linspace(0,1,5)
w = np.linspace(-1,1,10)
U = list(it.product(v,w))
n_controlspace = len(U)
pdf = multivariate_normal(mean=[0,0,0], cov=np.diag([0.04, 0.04, 0.004]))

time_step = 0.5
def lissajous(k):
    xref_start = 0
    yref_start = 0
    A = 2
    B = 2
    a = 2*np.pi/50
    b = 3*a
    T = np.round(2*np.pi/(a*time_step))
    k = k % T
    delta = np.pi/2
    xref = xref_start + A*np.sin(a*k*time_step + delta)
    yref = yref_start + B*np.sin(b*k*time_step)
    v = [A*a*np.cos(a*k*time_step + delta), B*b*np.cos(b*k*time_step)]
    thetaref = np.arctan2(v[1], v[0])
    return [xref, yref, thetaref]

traj = lissajous
cur_ref = []
for i in range(200):
    cur_ref.append(traj(i))


def error_dynamics(e, u,t, ref = cur_ref):
    '''
    : given current error, time, control, current reference and next reference
    : output -> Next error
    : Consider noise in the motion model. 
    '''
    G = np.array([[0.5 * np.cos(e[2] + ref[t][2]), 0], [0.5 * np.sin(e[2] + ref[t][2]), 0], [0, 0.5]])
    ref_diff = np.array([[ref[t][0] - ref[t+1][0]], [ref[t][1] - ref[t+1][1]], [ref[t][2] -  ref[t+1][2]]])
#     print(f'e shape : {e.shape}')
    next_error = e[:,None] + G @ np.array([[u[0]], [u[1]]]) + ref_diff
    next_error[2] = (next_error[2] + np.pi)%(2*np.pi) - np.pi
#     print(f'next error : {next_error}')
    next_x, next_y, next_th = next_error[0], next_error[1], next_error[2]
    ind_x = (next_x - e_x).argsort()[:5]
    ind_y = (next_y - e_y).argsort()[:5]
    ind_th = (next_th - th).argsort()[:5]    
    next_states, pr = [],[]
    for i in range(5):
        next_states.append(state_table[((t + 1)%50, e_x[ind_x[i]], e_y[ind_y[i]], th[ind_th[i]])])
        pr.append(pdf.pdf([e_x[ind_x[i]], e_y[ind_y[i]], th[ind_th[i]]]))
    pr = np.array(pr)
    pr /= np.sum(pr)
    return next_states, pr


def step_cost(e, u):
    '''
    : given the current error and the control
    : compute the step cost defined as the sum of tracking errors and control effort
    '''
    Q = np.eye(2)
    q = 1 
    R = np.eye(2)
    p = e[:2]
    th = e[-1]
    u = np.array([[u[0]],[u[1]]])
    return p.T @ Q @ p + q*(1 - np.cos(th))**2 + u.T @ R @ u

In [ ]:
L = np.zeros((n_states,n_controlspace))
for i in tqdm(range(n_states)):
    for j in range(n_controlspace):
        e = X[i][1:]
        L[i,j] = step_cost(np.array(e), U[j])

In [ ]:
import asyncio

def background(f):
    def wrapped(*args, **kwargs):
        return asyncio.get_event_loop().run_in_executor(None, f, *args, **kwargs)
    return wrapped

P = [0] * n_states
coords1,coords2, coords3 = [], [], []
data = []
@background
def create_P(i):
    e = np.array(X[i][1:])
    t = int(X[i][0]//0.5)
    p = np.zeros((n_controlspace, n_states))
    for j in range(n_controlspace):
        next_states, pr = error_dynamics(e,U[j],t)
#         print(f'probs : {pr}')
        for k in range(len(next_states)):
            p[j,next_states[k]] = pr[k]
            coords1.append(i)
            coords2.append(j)
            coords3.append(next_states[k])
            data.append(pr[k])    

L = [0] * n_states
@background
def create_L(i):
    e = X[i][1:]
    l = np.zeros(n_controlspace)
    for j in range(n_controlspace):
        l[j] = step_cost(np.array(e), U[j])
    L[i] = l

for i in tqdm(range(n_states)):
    create_P(i)
    create_L(i)

In [ ]:
coords = np.array([coords1, coords2, coords3])

In [ ]:
coords

In [ ]:
data

In [ ]:
from sparse import COO

In [ ]:
# coords = np.array([[0,1,2,3,5], [2,3,4,6,7], [0,1,4,5,6]])
# data = np.array([4,5,6,7,8])
p = COO(coords, data, shape = (n_states, n_controlspace,n_states))

In [ ]:
def value_iteration_matrix(V,pi,L,p):
    for i in tqdm(range(5000)):
        V_new = np.zeros_like(V)
        print(f'shape L : {L.shape}')
        print(f'shape : {np.sum(p @ V, axis = 2).shape}')
        Q = L + 0.98 * np.sum(p @ V, axis = 2)
        V_new = np.min(Q, axis = 1, keepdims=True)
#         print(f'shape V_new : {V_new.shape}')
        pi = np.argmin(Q, axis = 1)
        diff = np.linalg.norm(V_new - V, np.inf)
        print(f'diff : {diff}')
        if diff < 0.03 : 
            return V_new, pi
        else:
            V = V_new

In [ ]:
V = np.zeros((n_states, 1))
pi = np.zeros((n_states,2))
V_opt, pi_opt = value_iteration_matrix(V,pi,L,p)

In [ ]:
pi_opt